In [1]:
import numpy as np
import math 
import queue
import os
import neat
import visualize
import random
import matplotlib.pyplot as plt
import pickle

import GameManager
from GameManager import map_manager, Tile, Unit, Team

import Neat_AI
from Neat_AI import eval_genomes, run

import AI_modules
from AI_modules import no_ai, rand_ai, script_ai, neat_ai, script_performance

In [2]:
local_dir = os.path.abspath('')
config_path = os.path.join(local_dir, 'config-feedforward')

In [4]:
dimensions = (8,8)
manager = map_manager(dimensions)
#print(manager)

manager.setup_layouts_rand(layout_n=10, unit_count=5)
for i in range(len(manager.map_layouts)):
    manager.apply_map_layout(i)
    #print(manager)
    

In [ ]:
dimensions = (8,8)
units_per_side = 5
manager = map_manager(dimensions)

# for i in range(games_run):
#also resets map
manager.setup_rand(units_per_side)
print(manager)

while (manager.game_joever() == -1 and manager.turn_count < 8): #Turn Count limit may have to be modified
    for unit in manager.Teams[manager.curr_team].live_units:
        win_move = (0, 0)
        if manager.curr_team == 0:
            win_move = neat_ai(manager, unit, win_net)
        elif manager.curr_team == 1:
            win_move = script_ai(manager, unit)\
            #win_move = neat_ai(manager, unit, win_net)
        manager.move_unit(unit, win_move)
        print(manager)
    manager.Turn()

In [29]:
dimensions = (8,8)
units_per_side = 5
manager = map_manager(dimensions)

# for i in range(games_run):
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)
script_performance(manager, win_net, config)

In [3]:
from importlib import reload
reload(GameManager)
reload(AI_modules)
reload(Neat_AI)


from GameManager import map_manager, Tile, Unit, Team
from Neat_AI import eval_genomes, run, load_net
from AI_modules import no_ai, rand_ai, script_ai, neat_ai

In [4]:
random.seed(4123) #This is to set the random generated board starting positions
win_net, win_genome, stats = run(config_path, "New Self-Play test")


 ****** Running generation 0 ****** 



NameError: name 'manager' is not defined

In [234]:
win_net1 = win_net
stats1 = stats

In [267]:
#Restore a checkpoint to resume Run
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)
a
p = neat.checkpoint.Checkpointer.restore_checkpoint("./checkpoints/Global-Position-Eval-Func_vsScript_difference/neat-checkpoint-29")
winner = p.run(eval_genomes, 1)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

Mean genetic distance 1.323, standard deviation 0.442


In [172]:
#Restoring a best network for testing
net = load_net("./best/Relative-Position-Eval-Func-config", "./best/Relative-Position-Eval-Func-genome")
print(net)